In [2]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Đọc dữ liệu
file_path = "AQI-Air-Quality2.csv"
df = pd.read_csv(file_path)

# Đổi cột Date thành dang Datetime
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")
df = df.sort_values(by="Date")

# Chuyển đổi dữ liệu
aqi_columns = ["PM10_AQI", "NO2_AQI", "SO2_AQI", "CO_AQI", "O3_AQI"]
df[aqi_columns] = df[aqi_columns].apply(pd.to_numeric, errors='coerce')
df[aqi_columns] = df[aqi_columns].interpolate()

# Tạo đặc trưng với 5 ngày trước
n_lags = 5
for i in range(1, n_lags + 1):
    for col in ["PM2.5_AQI", "PM10_AQI", "NO2_AQI", "SO2_AQI", "CO_AQI", "O3_AQI"]:
        df[f"{col}_lag_{i}"] = df[col].shift(i)
df = df.dropna()

# Tách dữ liệu
features = [col for col in df.columns if "_lag_" in col]
targets = ["PM2.5_AQI", "PM10_AQI", "NO2_AQI", "SO2_AQI", "CO_AQI", "O3_AQI"]
X = df[features]
y = df[targets]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14)

# Huấn luyện mô hình
model = RandomForestRegressor(n_estimators=300, random_state=11)
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE(Sai số trung bình): {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

# Lưu mô hình
with open("aqi_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Mô hình đã được huấn luyện và lưu thành công!")


MAE(Sai số trung bình): 4.02
MSE: 50.44
R^2 Score: 0.66
Mô hình đã được huấn luyện và lưu thành công!


In [3]:
import pickle
import numpy as np
import tkinter as tk
from tkinter import messagebox

# Load mô hình đã huấn luyện
with open("aqi_model.pkl", "rb") as file:
    model = pickle.load(file)

# Hàm đánh giá chất lượng không khí
def evaluate_air_quality(aqi_values):
    total_aqi = max(aqi_values)
    if total_aqi <= 50:
        quality = "Tốt"
    elif total_aqi <= 100:
        quality = "Trung bình"
    elif total_aqi <= 150:
        quality = "Kém"
    elif total_aqi <= 200:
        quality = "Xấu"
    elif total_aqi <= 300:
        quality = "Rất xấu"
    else:
        quality = "Nguy hại"
    
    max_pollutant_index = np.argmax(aqi_values)
    main_pollutant = targets[max_pollutant_index]
    return total_aqi, quality, main_pollutant

# Hàm dự đoán AQI
def predict_aqi():
    try:
        input_data = [float(entries[key].get()) for key in input_keys]
        input_array = np.array(input_data).reshape(1, -1)
        prediction = model.predict(input_array)[0]
        
        total_aqi, quality, main_pollutant = evaluate_air_quality(prediction)
        
        result_text = "Dự đoán AQI cho ngày tiếp theo:\n"
        for i, feature in enumerate(targets):
            result_text += f"{feature}: {prediction[i]:.2f}\n"
        result_text += f"\nAQI Tổng: {total_aqi:.2f}\n"
        result_text += f"Chất lượng không khí: {quality}\n"
        result_text += f"Chất gây ô nhiễm chính: {main_pollutant}\n"
        
        result_label.config(text=result_text)
    except ValueError:
        messagebox.showerror("Lỗi", "Vui lòng nhập số hợp lệ cho tất cả các trường!")

# Tạo giao diện GUI
targets = ["PM2.5", "PM10", "NO2", "SO2", "CO", "O3"]
days = ["Day-5", "Day-4", "Day-3", "Day-2", "Day-1"]
input_keys = [f"{feature}_lag_{i+1}" for i in range(5) for feature in targets]

root = tk.Tk()
root.title("Dự đoán AQI")

frame = tk.Frame(root)
frame.pack(pady=10, padx=10)

# Tiêu đề cột
tk.Label(frame, text="").grid(row=0, column=0, padx=5, pady=5)
for j, feature in enumerate(targets):
    tk.Label(frame, text=feature, font=("Arial", 10, "bold")).grid(row=0, column=j+1, padx=5, pady=5)

entries = {}
for i, day in enumerate(days):
    tk.Label(frame, text=day, font=("Arial", 10, "bold")).grid(row=i+1, column=0, padx=5, pady=5)
    for j, feature in enumerate(targets):
        key = f"{feature}_lag_{i+1}"
        entry = tk.Entry(frame, width=10)
        entry.grid(row=i+1, column=j+1, padx=5, pady=5)
        entries[key] = entry

predict_button = tk.Button(root, text="Dự đoán", command=predict_aqi, font=("Arial", 10, "bold"))
predict_button.pack(pady=10)

result_label = tk.Label(root, text="", font=("Arial", 10), justify="left")
result_label.pack(pady=10)

root.mainloop()

C:\Users\dcsph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
